In [1]:
import os

import numpy as np
import pandas as pd
import joblib
from fuzzywuzzy import process
import spacy
import math
import re


In [2]:
nlp = spacy.load('en_core_web_md')

path = "E:/Notebooks/"

df = pd.read_csv('E:/Downloads/Merger - Sheet5 (2).csv')
category_names = df['CATEGORY'].unique()
brands = df['BRAND'].unique()
products = df['PRODUCT'].unique()
brand_vectors = {brand: nlp(brand).vector for brand in brands}

In [3]:
# Load the category prediction model and encoder
category_model = joblib.load("E:/Downloads/newp_svc_best_model.pkl")
category_encoder = joblib.load("E:/Downloads/newp_category_encoder.pkl")

E:\Notebooks\Pycharm\CatText\Lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
E:\Notebooks\Pycharm\CatText\Lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
E:\Notebooks\Pycharm\CatText\Lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your 

In [4]:
category_keywords = {
    'Baby Foods And Health Foods/ Drinks': ['BabyFood', 'Health', 'cereal', 'cornflakes', 'oats', 'museli'],
    'Cosmetics': ['Skin', 'cosmetic', 'skincream', 'beauty', 'baby', 'wax', 'lip', 'sunscreen', 'lipstick'],
    'Bath Soap/Liquids/Sanitizers': ['soap', 'shower', 'sanitizer', 'handwash', 'bodywash', 'facewash'],
    'Detergent/Washing Aids': ['detergent', 'bathroom', 'washroom', 'freshener', 'disinfectant', 'fabric', 'stiffener',
                               'stain', 'clean', 'sponge', 'scrub', 'broom', 'mop', 'wiper'],
    'Sanitary Napkins & Diapers': ['sanitary', 'pads', 'diapers', 'tampon'],
    'Shaving Care Products': ['shave', 'gillete', 'razor'],
    'Paper/Party': ['bags', 'foil', 'paper'],
    'Biscuits/Bakery Products': ['biscuit', 'bread', 'cake', 'cookies', 'wafer', 'rusk', 'toast'],
    'Chocolates And Confectionaries': ['candy', 'lollipop', 'jelly', 'chewing'],
    'Cold Drink/Juices/Drinks': ['juice', 'milkshake', 'crush', 'pulp', 'syrup', 'squash', 'cola', 'pepsi', 'drink'],
    'Deo/Talcum/Perfume': ['deo', 'talc', 'perfume', 'rollon'],
    'Hygiene': ['condom', 'intimate', 'nail'],
    'Dairy Products': ['dairy', 'dahi', 'lassi', 'shrikhand'],
    'Dental Care': ['dental', 'oral', 'floss', 'toothbrush', 'toothcare', 'toothpaste', 'tooth', 'mouthwash', 'tongue'],
    'Hair Care': ['shampoo', 'conditioner'],
    'General Merchandise': ['apparel', 'electric', 'plastic'],
    'Dryfruits': ['dryfruit', 'dates', 'badam', 'ajeer', 'kaju'],
    'Ready To Eat/Instant Food': ['instant', 'ready', 'noodles', 'pasta', 'soup'],
    'Farshan/Chikki/Mithai': ['namkeen', 'chikki', 'mithai', 'chips', 'farsan'],
    'Grocery': ['atta', 'flour', 'dal', 'grains', 'vegetables', 'rice', 'wheat'],
    'Upwas Special': ['upwas', 'farali'],
    'Frozen Foods': ['frozen', 'icecream', 'nuggets'],
    'Pooja Items': ['agarbatti', 'pooja', 'dhoop'],
    'Ghee/Vanaspati/Oils': ['ghee', 'vanaspati'],
    'Sugar/Salt': ['jaggery', 'salt', 'sugar'],
    'Masala And Spices': ['masala', 'spice', 'mukhwas', 'paste', 'puree'],
    'Jams/Ketchups/Spreads': ['essence', 'jam', 'sauce', 'chutney', 'ketchup', 'spread', 'vinegar', 'mayonnaise'],
    'Others': ['medicated', 'acidity'],
    'Mosquito Repellents And Pesticides': ['insect', 'pesticide', 'mosquito', 'repellent', 'rat'],
    'Papad/Pickles': ['papad', 'pickle', 'achaar'],
    'Shoe Care Products': ['shoe', 'boot', 'polish'],
    'Stationery': ['stationary', 'pen', 'book'],
    'Tea/Coffee': ['Tea', 'coffee', 'teabag']
}


In [5]:
default_subcategories = {
    "Dryfruits": "Dry Fruits",
    "Pooja Items": "Pooja Items",
    "Shaving Care Products": "Shaving Care Products",
    "Shoe Care Products": "Shoe Care Products",
    "Stationery": "Stationery",
    "Upwas Special": "Upwas Food"
}

In [6]:
def preprocess_text(texts):
    return [re.sub(r'[^A-Za-z]', ' ', text.lower()) if isinstance(text, str) else text for text in texts]

In [7]:
def predict_main_category(description, svc_model, encoder, category_names=category_names, fuzzy_match_threshold=90):
    description = str(description)
    # Preprocess the description
    processed_description = preprocess_text([description])[0]

    # Attempt fuzzy matching with the category names
    matched_category = None
    for word in processed_description.split():
        closest_match, score = process.extractOne(word, category_names)
        if score >= fuzzy_match_threshold:
            matched_category = closest_match
            break

    if matched_category:
        return matched_category, 0.75
    else:
        model_prediction = svc_model.predict([processed_description])
        prediction_confidence = svc_model.predict_proba([processed_description])
        decoded_prediction = encoder.inverse_transform(model_prediction)[0]
        confidence = max(prediction_confidence[0])  # Assuming the highest class probability
        return decoded_prediction, confidence

In [8]:
def predict_subcategory(description, main_category):
    if pd.isna(description):
        description = ""
    # If the main category has a default subcategory, return it
    if main_category in default_subcategories:
        return default_subcategories[main_category]

    # Otherwise, proceed with the model prediction
    safe_filename = main_category.replace(" ", "_").replace("/", "_and_")
    model_path = safe_filename + "_model.pkl"
    encoder_path = safe_filename + "_encoder.pkl"
    vectorizer_path = safe_filename + "_vectorizer.pkl"

    model = joblib.load(path + model_path)
    encoder = joblib.load(path + encoder_path)
    vectorizer = joblib.load(path + vectorizer_path)

    processed_description = preprocess_text([description])[0]
    vectorized_description = vectorizer.transform([processed_description])
    encoded_prediction = model.predict(vectorized_description)
    return encoder.inverse_transform(encoded_prediction)[0]


In [9]:
def predict_category_and_subcategory(description, model, encoder):
    main_category, confidence = predict_main_category(description, model, encoder)
    subcategory = predict_subcategory(description, main_category)
    return main_category, subcategory, confidence


In [10]:
def aggregate_and_predict(sales_data, description_column, quantity_column, price_column, brand_column='None'):
    # Remove rows where the product description (specified by description_column) is NaN
    sales_data = sales_data.dropna(subset=[description_column])

    # Ensure unique_data is a standalone DataFrame
    unique_data = sales_data.drop_duplicates(subset=description_column).copy()

    # Perform the predictions and store confidence
    predictions = unique_data[description_column].apply(lambda x: predict_category_and_subcategory(x, category_model, category_encoder))
    unique_data[['Predicted Category', 'Predicted Subcategory', 'Confidence']] = pd.DataFrame(predictions.tolist(), index=unique_data.index)

    # Mark low confidence predictions
    confidence_threshold = 0.35
    unique_data['LowConfidence'] = unique_data['Confidence'] < confidence_threshold

    # Merge back with the original sales data
    sales_data = sales_data.merge(unique_data[[description_column, 'Predicted Category', 'Predicted Subcategory', 'LowConfidence']],on=description_column, how='left')

    # Select only the required columns
    required_columns = [description_column, 'Predicted Category', 'Predicted Subcategory', 'LowConfidence', quantity_column, price_column]

    # Conditionally add columns if they are not 'None'
    if brand_column != 'None':
        required_columns.append(brand_column)

    sales_data = sales_data.loc[:, required_columns]

    return sales_data

In [11]:
def aggregate_sales_data(sales_data, group_column, quantity_column, price_column):
    """
    Aggregates sales data by a given column, based on Quantity and Price.
    """
    aggregation = {
        quantity_column: 'sum',
        price_column: 'sum'
    }

    aggregated_data = sales_data.groupby(group_column).agg(aggregation).reset_index()
    return aggregated_data


In [12]:
def find_top_and_least(data, column, n=5):
    """
    Finds the top and least n items based on a specific column.
    """
    if column in data.columns:
        top = data.nlargest(n, column)
        least = data.nsmallest(n, column)
        return top, least
    return None, None


In [13]:
def analyze_sales_data(data, group_column, quantity_column, price_column, analysis_type):
    """
    Analyzes sales data based on the provided group column and Quantity and Price.
    """
    stats = aggregate_sales_data(data, group_column, quantity_column, price_column)
    results = {}

    if stats is not None:
        results[f"top_{analysis_type}_by_quantity"], results[f"least_{analysis_type}_by_quantity"] = find_top_and_least(stats, quantity_column)
        results[f"top_{analysis_type}_by_value"], results[f"least_{analysis_type}_by_value"] = find_top_and_least(stats, price_column)

    return results


In [14]:
def analyze_category_sales(data, category_column, quantity_column, price_column):
    return analyze_sales_data(data, category_column, quantity_column, price_column, 'categories')


def analyze_subcategory_sales(data, subcategory_column, quantity_column, price_column):
    return analyze_sales_data(data, subcategory_column, quantity_column, price_column, 'subcategories')


def analyze_brand_sales(data, brand_column, quantity_column, price_column):
    return analyze_sales_data(data, brand_column, quantity_column, price_column, 'brands')


def analyze_item_sales(data, item_column, quantity_column, price_column):
    return analyze_sales_data(data, item_column, quantity_column, price_column, 'items')

In [15]:
sales_data = pd.read_excel("E:/Downloads/Chintan lunawad sales category.xlsx")
description_column='ITEM TYPE'
quantity_column='Total Sold Qty'
price_column='Sale Amount AfterTax'
brand_column='BRAND'

In [16]:
# Process and predict categories
sales_data = aggregate_and_predict(sales_data, description_column, quantity_column, price_column, brand_column)
print(sales_data)

          ITEM TYPE            Predicted Category Predicted Subcategory  \
0            COFFEE                    Tea/Coffee                Coffee   
1        BOURN VITA  Bath Soap/Liquids/Sanitizers             Face Wash   
2        BOURN VITA  Bath Soap/Liquids/Sanitizers             Face Wash   
3        BOURN VITA  Bath Soap/Liquids/Sanitizers             Face Wash   
4        BOURN VITA  Bath Soap/Liquids/Sanitizers             Face Wash   
...             ...                           ...                   ...   
2313   DHOOP STICKS                   Pooja Items           Pooja Items   
2314   DHOOP STICKS                   Pooja Items           Pooja Items   
2315   COTTON WICKS                   Pooja Items           Pooja Items   
2316          DHOOP                   Pooja Items           Pooja Items   
2317   DHOOP STICKS                   Pooja Items           Pooja Items   

      LowConfidence  Total Sold Qty  Sale Amount AfterTax  \
0             False             5.0   

In [17]:
# Filter for unique or low confidence predictions
unique_or_low_confidence_data = sales_data.drop_duplicates(
    subset=[description_column, 'Predicted Category', 'Predicted Subcategory'])
unique_or_low_confidence_data = unique_or_low_confidence_data[unique_or_low_confidence_data['LowConfidence']]

In [18]:
# Check if columns are 'None' and adjust the DataFrame operation
agg_columns = {}
if quantity_column != 'None':
    agg_columns[quantity_column] = 'sum'
if price_column != 'None':
    agg_columns[price_column] = 'sum'
if brand_column != 'None':
    agg_columns[brand_column] = 'sum'

# Include 'Predicted Category' and 'Predicted Subcategory' using mode
agg_columns['Predicted Category'] = lambda x: x.mode()[0] if not x.mode().empty else np.nan
agg_columns['Predicted Subcategory'] = lambda x: x.mode()[0] if not x.mode().empty else np.nan

if agg_columns:  # Check if there are columns to aggregate
    grouped_data = sales_data.groupby(description_column).agg(agg_columns)
    # Reset the index to make grouped columns regular columns
    grouped_data = grouped_data.reset_index()


# Assuming quantity_column and price_column are obtained from user input
category_results = analyze_category_sales(grouped_data, 'Predicted Category', quantity_column, price_column)
subcategory_results = analyze_subcategory_sales(grouped_data, 'Predicted Subcategory', quantity_column,
                                                price_column)
brand_results = analyze_brand_sales(grouped_data, brand_column, quantity_column, price_column)
item_results = analyze_item_sales(grouped_data, description_column, quantity_column,
                                  price_column)  # column_name is the identifier for items

# Combine all results
# results = {**category_results, **subcategory_results, **brand_results, **item_results}
# print(results)
# Assuming category_results, subcategory_results, brand_results, item_results are DataFrames

results = {
    'categories': {
        'data': category_results,
        'is_empty': not category_results
    },
    'subcategories': {
        'data': subcategory_results,
        'is_empty': not subcategory_results
    },
    'brands': {
        'data': brand_results,
        'is_empty': not brand_results
    },
    'items': {
        'data': item_results,
        'is_empty': not item_results
    }
}

In [19]:
print(results)

{'categories': {'data': {'top_categories_by_quantity':                 Predicted Category  Total Sold Qty  Sale Amount AfterTax
9           Detergent/Washing Aids           618.0              29320.66
15                       Hair Care           527.0              12505.12
12             General Merchandise           498.0              36362.00
2         Biscuits/Bakery Products           413.0              12904.05
3   Chocolates And Confectionaries           411.0               5239.00, 'least_categories_by_quantity':                       Predicted Category  Total Sold Qty  Sale Amount AfterTax
16                               Hygiene             2.0                  50.0
0    Baby Foods And Health Foods/ Drinks             6.0                 587.0
19  Mosquito Repellents And  Pesticides              6.0                 302.0
22                           Pooja Items             9.0                 195.4
25                 Shaving Care Products             9.0                 520.0,

In [22]:
# Assuming 'results' is your main dictionary
categories_data = results['categories']['data']

# Accessing the DataFrame for top categories by quantity
top_categories_df = categories_data['top_categories_by_quantity']

# Accessing the 'Predicted Category' column
predicted_categories = top_categories_df['Predicted Category']

# If you want it as a list
predicted_category_list = predicted_categories.tolist()

print(predicted_category_list)


['Detergent/Washing Aids', 'Hair Care', 'General Merchandise', 'Biscuits/Bakery Products', 'Chocolates And Confectionaries']


In [23]:
top_categories_list = results['categories']['data']['top_categories_by_quantity'].to_dict(orient='records')
for row in top_categories_list:
    print(row['Predicted Category'])

Detergent/Washing Aids
Hair Care
General Merchandise
Biscuits/Bakery Products
Chocolates And Confectionaries
